# M2S13.01: Exploring alternative clustering methods

## 01.1. Library and files loading

In [ ]:
library(Seurat)
library(clustree)
library(cluster)
library(ggplot2)
library(dplyr)

# BiocManager::install('glmGamPoi')
# devtools::install_github("immunogenomics/presto")

In [ ]:
count_matrix <- read.csv("count_matrix.csv", row.names = 1, check.names = FALSE)
metadata <- read.csv("metadata.csv", row.names = 1)

## 01.2. Quick Seurat workflow (see previous classes)

In [ ]:
# Create a Seurat object
seurat_object <- CreateSeuratObject(counts = count_matrix, meta.data = metadata, min.cells = 3, min.features = 200)

# Standard workflow
seurat_object <- SCTransform(seurat_object, verbose = FALSE)
seurat_object <- RunPCA(seurat_object, features = VariableFeatures(object = seurat_object))
seurat_object <- FindNeighbors(seurat_object, dims = 1:30)
seurat_object <- FindClusters(seurat_object, resolution = 0.5)
seurat_object <- RunUMAP(seurat_object, dims = 1:30)

In [ ]:
DimPlot(seurat_object, reduction = "umap", group.by = "Selection", label = TRUE, label.box = TRUE, repel = TRUE) + NoLegend()
DimPlot(seurat_object, reduction = "umap", group.by = "Location", label = TRUE, label.box = TRUE, repel = TRUE) + NoLegend()
DimPlot(seurat_object, reduction = "umap", label = TRUE, label.box = TRUE, repel = TRUE) + NoLegend()

## 01.3. Selecting optimal clustering resolution

In [ ]:
resolutions <- seq(from = 0.2, to = 2, by = 0.2)
seurat_object <- FindClusters(seurat_object, resolution = resolutions)

### _Ad oculus_: cluster tree

In [ ]:
# Draw cluster tree with clustree to assess stability across resolutions
clustree(seurat_object, prefix = "SCT_snn_res.")

### Metrics-based: silhouette score

In [ ]:
pca_embeddings <- Embeddings(seurat_object, reduction = "pca")
dist_matrix <- dist(pca_embeddings)

# Compute silhouette scores for each clustering resolution
sil_scores <- sapply(resolutions, function(res) {
  col_name <- paste0("SCT_snn_res.", res)
  cluster_ids <- as.numeric(as.factor(seurat_object@meta.data[[col_name]]))
  if (length(unique(cluster_ids)) < 2) {
    return(NA) 
  }
  sil <- silhouette(cluster_ids, dist_matrix)
  if (is.null(dim(sil))) {
    return(NA)
  }
  mean(sil[, "sil_width"])
})

sil_df <- data.frame(Resolution = resolutions, Avg_Silhouette = sil_scores)
print(sil_df)

ggplot(sil_df, aes(x = Resolution, y = Avg_Silhouette)) +
  geom_line() +
  geom_point() +
  theme_minimal() +
  labs(title = "Average Silhouette Score Across Clustering Resolutions",
       y = "Average Silhouette Width") +
  scale_x_continuous(breaks = sil_df$Resolution)

In [ ]:
seurat_object <- FindClusters(seurat_object, resolution = 0.6)

In [ ]:
DimPlot(seurat_object, reduction = "umap", label = TRUE, label.box = TRUE, repel = TRUE) + NoLegend()

In [ ]:
FeaturePlot(seurat_object, features = "PTPRC")
FeaturePlot(seurat_object, features = "THY1")
FeaturePlot(seurat_object, features = "OLIG1")

In [ ]:
markers <- FindMarkers(seurat_object, ident.1 = "0", ident.2 = "2", min.diff.pct = 0.25)

In [ ]:
markers %>% filter(abs(avg_log2FC) > 0.5, p_val_adj < 0.05) %>% arrange(desc(avg_log2FC))

## 01.4. Exporting as AnnotData object for future analysis

In [ ]:
library(SeuratDisk)

In [ ]:
SaveH5Seurat(seurat_object, filename = "01_object.h5Seurat")

In [ ]:
Convert("01_object.h5Seurat", dest = "h5ad", overwrite = TRUE)

In [ ]:
saveRDS(seurat_object, "01_seurat_obj.Rds")